# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [55]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

import pickle

[nltk_data] Downloading package punkt to /home/jc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('MessageTable', engine)
X = df['message']
Y = df.drop(columns=['id', 'message','original','genre' ])

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = word_tokenize(text)
    
    # Remove Stopwords
    tokens = [w for w in tokens if w not in stopwords.words('english')]
    
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [28]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, np.array(Y), random_state=42)

In [30]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [31]:
import sklearn
sklearn.__version__

'0.21.2'

In [32]:
y_pred = pipeline.predict(X_test)

In [33]:
y_test.shape

(6554, 36)

In [34]:
y_pred.shape

(6554, 36)

In [35]:
category_names = list(Y.columns.values)

In [36]:
category_names

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [37]:
y_test

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 1, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [38]:
y_pred

array([[0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [39]:
for category in range(len(category_names)):
    print('category: {}'.format(category_names[category]))
    print(classification_report(y_test[:, category], y_pred[:, category]))

#print(classification_report(y_test.values, y_pred))
#print(classification_report(y_test, y_pred, target_names=Y.columns.values))

category: related
              precision    recall  f1-score   support

           0       0.61      0.39      0.48      1563
           1       0.82      0.92      0.87      4944
           2       0.32      0.15      0.20        47

    accuracy                           0.79      6554
   macro avg       0.58      0.49      0.52      6554
weighted avg       0.77      0.79      0.77      6554

category: request
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      5443
           1       0.82      0.41      0.55      1111

    accuracy                           0.88      6554
   macro avg       0.86      0.70      0.74      6554
weighted avg       0.88      0.88      0.87      6554

category: offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.5

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6219
           1       0.46      0.03      0.06       335

    accuracy                           0.95      6554
   macro avg       0.70      0.52      0.52      6554
weighted avg       0.93      0.95      0.93      6554

category: direct_report
              precision    recall  f1-score   support

           0       0.85      0.98      0.91      5282
           1       0.73      0.26      0.39      1272

    accuracy                           0.84      6554
   macro avg       0.79      0.62      0.65      6554
weighted avg       0.82      0.84      0.81      6554



/home/jc/.conda/envs/udacity/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [40]:
Y['related'].unique()

array([1, 0, 2])

In [41]:
print('Accuracy: {}'.format(np.mean(y_test == y_pred)))

Accuracy: 0.9437451259620927


### 6. Improve your model
Use grid search to find better parameters. 

In [44]:
parameters = {
    'tfidf__use_idf':[True, False],
    'clf__estimator__n_estimators': [10,20,30,40],
    'clf__estimator__max_depth': [3, None],
    'clf__estimator__min_samples_split': [2,3,10],
    'clf__estimator__criterion' : ['gini', 'entropy']
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=3, verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [45]:
tuned_model = cv.fit(X_train, y_train)

Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=3, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=3, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.189, total=  37.0s
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=3, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   37.0s remaining:    0.0s


[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=3, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.193, total=  37.5s
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=3, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


...

[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.231, total=  51.7s
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.237, total=  51.7s
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, score=0.232, total= 1.1min
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimat

[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=10, clf__estimator__n_estimators=20, score=0.240, total=  57.3s
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=10, clf__estimator__n_estimators=20 
[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=10, clf__estimator__n_estimators=20, score=0.237, total=  57.5s
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=10, clf__estimator__n_estimators=30 
[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=10, clf__estimator__n_estimators=30, score=0.233, total= 1.1min
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=10, clf__estimator__n_estimators=30 
[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__e

[CV]  clf__estimator__criterion=entropy, clf__estimator__max_depth=3, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=30, score=0.193, total=  38.5s
[CV] clf__estimator__criterion=entropy, clf__estimator__max_depth=3, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=30 
[CV]  clf__estimator__criterion=entropy, clf__estimator__max_depth=3, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=30, score=0.201, total=  38.5s
[CV] clf__estimator__criterion=entropy, clf__estimator__max_depth=3, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=40 
[CV]  clf__estimator__criterion=entropy, clf__estimator__max_depth=3, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=40, score=0.189, total=  39.1s
[CV] clf__estimator__criterion=entropy, clf__estimator__max_depth=3, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=40 
[CV]  clf__estimator__criterion=entropy, clf__estimator__max_depth=3, clf__estimat

[CV]  clf__estimator__criterion=entropy, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=40, score=0.243, total= 1.7min
[CV] clf__estimator__criterion=entropy, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=40 
[CV]  clf__estimator__criterion=entropy, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=40, score=0.235, total= 1.7min
[CV] clf__estimator__criterion=entropy, clf__estimator__max_depth=None, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10 
[CV]  clf__estimator__criterion=entropy, clf__estimator__max_depth=None, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, score=0.219, total=  51.2s
[CV] clf__estimator__criterion=entropy, clf__estimator__max_depth=None, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10 
[CV]  clf__estimator__criterion=entropy, clf__estimator__max_dep

[Parallel(n_jobs=1)]: Done 144 out of 144 | elapsed: 130.4min finished


In [46]:
tuned_model.best_params_

{'clf__estimator__criterion': 'gini',
 'clf__estimator__max_depth': None,
 'clf__estimator__min_samples_split': 3,
 'clf__estimator__n_estimators': 40}

In [52]:
tuned_model.best_estimator_

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

In [50]:
y_pred_tuned = tuned_model.predict(X_test)

In [51]:
print('Accuracy: {}'.format(np.mean(y_test == y_pred_tuned)))

Accuracy: 0.9473222459566677


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [58]:
pickle.dump(tuned_model.best_estimator_, open('./model.pickle', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.